In [7]:
from pathlib import Path

In [22]:
src_symlink = Path().resolve() / 'src'
if not src_symlink.is_symlink():
    src_symlink.symlink_to(src_symlink.parents[1] / 'src')

In [1]:
%load_ext autoreload

In [246]:
%autoreload 1

In [247]:
%aimport src.preprocessing

In [248]:
from src.preprocessing import Cleaner

In [284]:
import difflib
from collections import Counter, Mapping
import math
from itertools import zip_longest
from Levenshtein import median, median_improve, ratio
import string
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt

In [325]:
cleaner = Cleaner(lowercase=False, table=None)
cleaner('Aa.,;#fheekJK### \r\n karim')

'Aa.,;#fheekJK### \r\n karim'

In [231]:
repo_dir = Path('/home/karimlulu/repos/fuzzygrouping')
data_dir = repo_dir / 'data'
filename = 'report.xlsx'

In [365]:
df = pd.read_excel(data_dir / filename)
words = df.loc[df.Month<=11, 'Reseller name'].sample(frac=0.99).values

In [33]:
def find_indices(list_, element):
    rez = []
    offset = -1
    while True:
        try:
            offset = list_.index(element, offset+1)
            rez.append(offset)
        except ValueError:
            return rez

In [366]:
# data = ['евтушенко', "евтушенко сергей", "Лозовая .", "Лозовая Мария", "лозовая",
#          "сергей евтушенко", "Евтушенкоо", "Евтушенко Сергей", "мария лозовая",
#         "United states", "United States", "Ukraine", "united states of americe", 
#         "United States of America"]
data = words
print("Input: {0}\n".format(data))
data = [cleaner(el) for el in data]
temp_vocab = sorted(data, key=len, reverse=True)
mapper = {}
while temp_vocab:
    word = temp_vocab[0]
    temp = [el[0] for el in process.extractBests(query=word, choices=temp_vocab[1:], processor=cleaner, scorer=fuzz.token_set_ratio, limit=2**64, score_cutoff=80)]
    #difflib.get_close_matches(word, temp_vocab[1:], n = 2**64, cutoff=0.5)
   
    group = [word, *temp]
    most_common = mode(group)
    mapper.update(list(zip_longest(set(temp), [word], fillvalue=word)))
    temp_vocab = [el for el in temp_vocab if el not in group]

ignore = ['Овчаренко - Филимонова']
for key,value in mapper.items():
    if (value in ignore) and (key!=value):
        mapper[key] = key
        value = key

mapper = {k:v for k,v in mapper.items() if k!=v}
for i,el in enumerate(data):
    data[i] = mapper.get(el, el)
#print("Output: {0}".format(data))

Input: ['Die Zwergperten München' 'ООО «Оникс»' 'ФОП Тараканов' ...,
 'Baby-Starke / Dirk Schulze eK' 'Автодіти' 'Baby Centrum']



In [367]:
mapper

{'4 mama': 'Mama i Tata',
 '7 Kinder': 'Kinder Centrum',
 '9 LUNAS': 'NUEVE LUNAS',
 'ABC': 'Małe ABC',
 'ALI BEBE': 'ALMA BEBE',
 'Agata': 'Agatka',
 'Ali bebe': 'ALMA BEBE',
 'Altrak': 'Alltrak',
 'Anex-shop': 'Планета - Anex-Shop',
 'BABY BOOM': 'NEREA GOMEZ BABY BOOM',
 "BABY'S...SHOP": 'Krümel-Shop rund ums Baby & Kind',
 'BAMBINOS': 'BAMBINOS VIGO CAMELIAS',
 'BBELANDIA': 'Baby Landia',
 'BBLANDIA': 'Baby Landia',
 'BEBECENTER': 'BEBE CENTER',
 'BEBES MERIDA': 'Bebes Merida',
 'Baby & Kind Pilhofer': 'Raumausstattung, Baby & Kind, Helmut Pilhofer',
 'Baby Boom': 'NEREA GOMEZ BABY BOOM',
 'Baby Land': 'Baby Land Crimmitschau',
 'Baby Lux': 'Baby Blue',
 'Baby Shop': 'Krümel-Shop rund ums Baby & Kind',
 'Baby world': 'Babys Wold',
 'Baby&Shop': 'Krümel-Shop rund ums Baby & Kind',
 'Baby+': 'Babyland  Mecklenburg -Happy Baby- der große Fachmarkt GmbH',
 'BabyOne Binzen': 'BabyOne Waiblingen',
 'BabyOne Heilbronn': 'BabyOne Heidelberg',
 'BabyOne Senden': 'BabyOne Leinfelden',
 'Baby

In [36]:
def mode(group):
    return Counter(group).most_common(1)[0][0]

In [37]:
def median_string(group):
    return median(group)

In [265]:
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance


dist_func = distance.levenshtein
words = np.asarray(words) #So that indexing with a list will work
sample = np.asarray(sorted([el for el in words if 'poderova' in str(el)]))
lev_similarity = -10*np.array([[dist_func(w1,w2) for w1 in sample] for w2 in sample])


affprop = AffinityPropagation(affinity="precomputed", damping=0.6, max_iter=200)
print('after lev')
affprop.fit(lev_similarity)

for cluster_id in np.unique(affprop.labels_):
    exemplar = sample[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(sample[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = "; ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

after lev


AffinityPropagation(affinity='precomputed', convergence_iter=15, copy=True,
          damping=0.6, max_iter=200, preference=None, verbose=False)

 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *K. Krupoderova:* K. Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia Krupoderova:* Ksenia Krupoderova
 - *Ksenia K

In [279]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

def homogenized(strings):
    lengths = [len(s) for s in strings]
    n = max(lengths)
    for s in strings:
        k = len(s)
        yield [k] + [ord(c) for c in s] + [0] * (n - k)

def dehomogenized(points):
    for p in points:
        k = int(p[0])
        yield ''.join(chr(int(x)) for x in p[1:k+1])

def mylev(u, v):
    return dist_func(*list(dehomogenized((u, v))))

points = np.array(list(homogenized(sample)))

lev = pairwise_distances(points, metric=mylev, n_jobs=1)